In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
blok_size = 64 #блок - это длинна секции вектора, а батч - несколько блоков
batch_size = 128
max_iters = 3_000
# eval_iterval = 500
learning_rate = 3e-4
eval_iters = 100

n_embd = 384
n_layer = 4
n_head = 4
dropout = 0.2

cuda


In [14]:
wizard_of_oz_book_data_path = 'oz_book\wizard_of_oz.txt'

In [15]:
with open(wizard_of_oz_book_data_path, 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [16]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encoded_hello = encode('hello')
# decoded_hello = decode(encoded_hello)
# print(decoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [17]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - blok_size, (batch_size,))
#     print(ix)
    x = torch.stack([data[i:i+blok_size] for i in ix])
    y = torch.stack([data[i+1:i+blok_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
#print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[68, 67, 60,  ..., 64, 58, 57],
        [76, 61, 68,  ..., 58, 58, 71],
        [33,  1, 55,  ...,  1, 58, 75],
        ...,
        [ 1, 76, 54,  ..., 69, 54, 73],
        [54, 67, 56,  ...,  1, 66, 68],
        [ 5, 72,  1,  ..., 54, 73,  1]], device='cuda:0')
targets:
tensor([[67, 60,  1,  ..., 58, 57,  1],
        [61, 68,  1,  ..., 58, 71,  1],
        [ 1, 55, 58,  ..., 58, 75, 58],
        ...,
        [76, 54, 72,  ..., 54, 73, 61],
        [67, 56, 58,  ..., 66, 68, 75],
        [72,  1, 65,  ..., 73,  1, 68]], device='cuda:0')


In [18]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [25]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(blok_size, blok_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Входной размер (batch, time-step, channels)
        # Выходной размер (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T]) == 0, float('-inf')
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj =  nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(), 
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Блок трансформер"""
    
    def __init__(self, n_embd, n_head):
        # n_embd - размерность эмбендинга, n_head - число заголовков (heads) которое нам хочется
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x 

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embending_table = nn.Embedding(vocab_size, n_embd)
        self.position_embending_table = nn.Embedding(blok_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Финальный нормализующий слой
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self.__init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(model.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets=None):
        logits = self.token_embending_table(index)
        B, T, C = logits.shape
        
        # индексы и таргеты оба (B, T) тензоры типа интеджер
        tok_emb = self.token_embending_table(idex) # (B, T, C)
        pos_emb = self.position_embending_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        
        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # индексы (B, T) это массивы в текущем контексте
        for _ in range(max_new_tokens):
            # получим предсказания
            logits, loss = self.forward(index)
            # фокус только на последний шаш (time step)
            logits = logits[:, -1, :] # становятся (B, C)
            # Применяем софтмакс чтобы получить вероятности
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Отрезок из распределения
            index_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # добавим индекс отрезка в запущенную секцию 
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


jPP.CvHRdo"2,n,xjuon)CgA[﻿ 'CQoyHRgIQ3C](WNyHpF7ia.8)w-8e8ayK7HsL;l7DM[m(M9tjjY_wY9,e
pEvHzNzOSY_f9p54:::W_uANQCF*:xE0w?'xywC2F7WtD3Y:f-s?EfDw-"tgNN2,h6:152wtuWb:hw2*N34.k&GW[mlL*,EH8Z Z'eUS7h*ap:Gf-
0_UBxkffa*zOhbRtHdp5?7ldGRcms"t7?X'?I3HS772,(Ie-
7q(4n)(0Yvm0i'TJPk6;VsX-Yd:14V[3_NNQ,1WyquamUS2e!tGYFQ8fdmkk*aQ1WWy'c(F7o4?HvN DS4KTGYGLyl6,K?UFN[KRi!&C)e-VmXaCCVxtg?Mz,1_5-mhb'TgLAmU
vvn. e-ND* -!QsuBVsk_Y_UNuHjsuW;'SnD8LAe9(,mxYFKy6C.MEb9)B&B?Irh4eYLppG09EPDb"th):f.w)n8h(,GEdLfT&7EeIHDWYpfx(ndEPI


In [26]:
# Создадим PyTorch оптимайзер
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iteration in range(max_iters):
    if iteration % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iteration}, train loss: {losses['train']:.4f}, validation loss: {losses['val']:.4f}")

    # отрезок из бача датасета
    xb, yb = get_batch('train')
    
    # Расчет потерь
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.8619, validation loss: 4.8491
step: 100, train loss: 4.8158, validation loss: 4.8040
step: 200, train loss: 4.7699, validation loss: 4.7563
step: 300, train loss: 4.7235, validation loss: 4.7136
step: 400, train loss: 4.6799, validation loss: 4.6681
step: 500, train loss: 4.6347, validation loss: 4.6243
step: 600, train loss: 4.5900, validation loss: 4.5819
step: 700, train loss: 4.5484, validation loss: 4.5385
step: 800, train loss: 4.5049, validation loss: 4.4957
step: 900, train loss: 4.4638, validation loss: 4.4555
step: 1000, train loss: 4.4208, validation loss: 4.4141
step: 1100, train loss: 4.3832, validation loss: 4.3724
step: 1200, train loss: 4.3428, validation loss: 4.3321
step: 1300, train loss: 4.3012, validation loss: 4.2926
step: 1400, train loss: 4.2606, validation loss: 4.2569
step: 1500, train loss: 4.2234, validation loss: 4.2174
step: 1600, train loss: 4.1855, validation loss: 4.1777
step: 1700, train loss: 4.1498, validation loss: 4.1423
step